In [3]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
import math
import keras.backend as K
import csv
from tqdm import tqdm
from keras.models import Model
import copy

In [2]:
train = ImageDataGenerator(rescale=1/255)
test = ImageDataGenerator(rescale=1/255)
batch_size = 32
train_dataset = train.flow_from_directory("ImageDataset/TrainSet/",
                                          target_size=(30,150),
                                          batch_size = batch_size,
                                          class_mode = 'binary')
                                         
test_dataset = test.flow_from_directory("ImageDataset/TestSet/",
                                          target_size=(30,150),
                                          batch_size = batch_size,
                                          class_mode = 'binary')

Found 11668 images belonging to 2 classes.
Found 4000 images belonging to 2 classes.


In [3]:
model = tf.keras.models.Sequential()

model.add(keras.layers.Conv2D(32,(3,3),activation='relu',input_shape=(30,150,3)))
model.add(keras.layers.MaxPool2D(2,2))
# model.add(keras.layers.BatchNormalization())


model.add(keras.layers.Conv2D(64,(3,3),activation='relu'))
model.add(keras.layers.MaxPool2D(2,2))
# model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Dropout(0.2))

model.add(keras.layers.Conv2D(128,(3,3),activation='relu'))
model.add(keras.layers.MaxPool2D(2,2))
# model.add(keras.layers.BatchNormalization())


model.add(keras.layers.Flatten())


model.add(keras.layers.Dense(256,activation='relu'))
# model.add(keras.layers.BatchNormalization())


model.add(keras.layers.Dense(1,activation='sigmoid'))

In [4]:
model.compile(optimizer='adam',loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 148, 32)       896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 74, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 12, 72, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 36, 64)         0         
_________________________________________________________________
dropout (Dropout)            (None, 6, 36, 64)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 4, 34, 128)        73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 2, 17, 128)        0

In [5]:

steps_per_epoch = len(train_dataset)

model.fit(train_dataset,
         steps_per_epoch = steps_per_epoch,
         epochs = 10,
         validation_data = test_dataset
       
         )

Epoch 1/10
365/365 [==============================] - 40s 103ms/step - loss: 0.6293 - accuracy: 0.6342 - val_loss: 0.5201 - val_accuracy: 0.7505
Epoch 2/10
365/365 [==============================] - 33s 91ms/step - loss: 0.4774 - accuracy: 0.7824 - val_loss: 0.4607 - val_accuracy: 0.7940
Epoch 3/10
365/365 [==============================] - 32s 89ms/step - loss: 0.4136 - accuracy: 0.8143 - val_loss: 0.4477 - val_accuracy: 0.8052
Epoch 4/10
365/365 [==============================] - 33s 89ms/step - loss: 0.3697 - accuracy: 0.8419 - val_loss: 0.4213 - val_accuracy: 0.8135
Epoch 5/10
365/365 [==============================] - 33s 90ms/step - loss: 0.3349 - accuracy: 0.8576 - val_loss: 0.4087 - val_accuracy: 0.8223
Epoch 6/10
365/365 [==============================] - 33s 90ms/step - loss: 0.3019 - accuracy: 0.8723 - val_loss: 0.4191 - val_accuracy: 0.8253
Epoch 7/10
365/365 [==============================] - 33s 90ms/step - loss: 0.2740 - accuracy: 0.8854 - val_loss: 0.4497 - val_accuracy

### Test Set

In [6]:
entries = os.listdir('ImageDataset/TestSet/nonword/')
nonwords = []
for e in entries:
    nonwords.append(e)

    
entries = os.listdir('ImageDataset/TestSet/word/')
words = []
for e in entries:
    words.append(e)

In [7]:
model.pop()
model.compile()
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 148, 32)       896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 74, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 12, 72, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 36, 64)         0         
_________________________________________________________________
dropout (Dropout)            (None, 6, 36, 64)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 4, 34, 128)        73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 2, 17, 128)        0

In [8]:
prob = []
all_test_set = words+nonwords
all_feature_vec = []
for e in tqdm(nonwords):
    filename = "ImageDataset/TestSet/nonword/" + e
    img = image.load_img(filename,target_size=(30,150))
 
    Y = image.img_to_array(img)
    X = np.expand_dims(Y,axis=0)
    X = X/255
    val = model.predict(X)
    all_feature_vec.append(val.reshape(256))

for e in tqdm(words):
    filename = "ImageDataset/TestSet/word/" + e
    img = image.load_img(filename,target_size=(30,150))
 
    Y = image.img_to_array(img)
    X = np.expand_dims(Y,axis=0)
    X = X/255
    val = model.predict(X)
    all_feature_vec.append(val.reshape(256))


100%|██████████| 2000/2000 [01:04<00:00, 30.90it/s]


### Train Set

In [6]:
model.pop()
model.compile()
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 148, 32)       896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 74, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 12, 72, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 36, 64)         0         
_________________________________________________________________
dropout (Dropout)            (None, 6, 36, 64)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 4, 34, 128)        73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 2, 17, 128)        0

In [7]:
entries = os.listdir('ImageDataset/TrainSet/nonword/')
nonwords = []
for e in entries:
    nonwords.append(e)


    
entries = os.listdir('ImageDataset/TrainSet/word/')
words = []
for e in entries:
    words.append(e)

In [8]:
prob = []
all_test_set = words+nonwords
all_feature_vec = []
for e in tqdm(nonwords):
    filename = "ImageDataset/TrainSet/nonword/" + e
    img = image.load_img(filename,target_size=(30,150))
 
    Y = image.img_to_array(img)
    X = np.expand_dims(Y,axis=0)
    X = X/255
    val = model.predict(X)
    all_feature_vec.append(val.reshape(256))
    
for e in tqdm(words):
    filename = "ImageDataset/TrainSet/word/" + e
    img = image.load_img(filename,target_size=(30,150))
 
    Y = image.img_to_array(img)
    X = np.expand_dims(Y,axis=0)
    X = X/255
    val = model.predict(X)
    all_feature_vec.append(val.reshape(256))



100%|██████████| 6668/6668 [03:33<00:00, 31.24it/s]


In [9]:
print(len(all_feature_vec)) #rows
print(len(all_feature_vec[0])) #columns

11668
256


## Read Data From TEST File

In [12]:
data = []
acc = []
rt = []
prt = []
nrt = []
with open("Datasets/test_dataset_rt_acc.csv", 'r') as file:
    reader = csv.reader(file)
    for row in reader:
        line = []
        line.append(row[0])
        line.append(float(row[4]))
        acc.append(float(row[4]))
        line.append(float(row[2]))
        rt.append(float(row[2]))
        if float(row[4]) == 0:
            nrt.append(float(row[2]))
        else:
            prt.append(float(row[2]))
        data.append(line)

## Read Data From TRAIN File

In [11]:
data = []
acc = []
rt = []
prt = []
nrt = []
with open("Datasets/train_dataset_prob.csv", 'r') as file:
    reader = csv.reader(file)
    for row in reader:
        line = []
        line.append(row[0])
        line.append(float(row[4]))
        acc.append(float(row[4]))
        line.append(float(row[2]))
        rt.append(float(row[2]))
        if float(row[4]) == 0:
            nrt.append(float(row[2]))
        else:
            prt.append(float(row[2]))
        data.append(line)

IndexError: list index out of range

## Stan

In [13]:
import nest_asyncio
nest_asyncio.apply()
import pystan
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

### Neural Network Weights

In [12]:
stan_model= """
functions {
  vector nn_predict(matrix x, vector d_t_h, real y_bias) {
    int N = rows(x);
    vector[N] output_layer;
    
    output_layer = inv_logit(x * d_t_h + y_bias);
    return(output_layer);
  }
}

data {
  int N; // Number of training samples
  int P; // Number of predictors (features)
  matrix[N, P] x; // Feature data
  vector[N] y; // Outcome

  int N_test; // Number of test samples
  matrix[N_test, P] x_test; // Test predictors
}

transformed data {
}

parameters {
  vector[P] data_to_hidden_weights; // Data -> Hidden 1
  real y_bias; // Bias. 
  real<lower=0> sigma;
}

transformed parameters {
  vector[N] output_layer;

  output_layer = nn_predict(x, data_to_hidden_weights, y_bias);

}

model {
  // Priors
  to_vector(data_to_hidden_weights) ~ std_normal();

  y_bias ~ std_normal();

  sigma ~ std_normal();


  y ~ normal(output_layer, sigma);
}
"""

In [1]:
stan_model = """
functions {
  vector[] nn_predict(matrix x, matrix d_t_h, matrix[] h_t_h, matrix h_t_d, row_vector[] hidden_bias, row_vector y_bias) {
    int N = rows(x);
    int n_H = cols(d_t_h);
    int H = size(hidden_bias);
    int num_labels = cols(y_bias) + 1;
    matrix[N, n_H] hidden_layers[H];
    vector[num_labels] output_layer_logit[N];
    vector[N] ones = rep_vector(1., N);

    hidden_layers[1] = inv_logit(x * d_t_h + ones * hidden_bias[1]);
    for(h in 2:H) {
      hidden_layers[h] = inv_logit(hidden_layers[h-1] * h_t_h[h - 1] + ones * hidden_bias[h]);
    }
    for(n in 1:N) {
      output_layer_logit[n, 1] = 0.0;
      output_layer_logit[n, 2:num_labels] = (hidden_layers[H, n] * h_t_d + y_bias)';
    }
    return(output_layer_logit);
  }
}

data {
  int N; // Number of training samples
  int P; // Number of predictors (features)
  matrix[N, P] x; // Feature data
  int labels[N]; // Outcome labels



transformed data {
  int num_labels = 2; // How many labels are there
}

parameters {
  matrix[P, n_H] data_to_hidden_weights; // Data -> Hidden 1
  matrix[n_H, n_H] hidden_to_hidden_weights[H - 1]; // Hidden[t] -> Hidden[t+1]
  matrix[n_H, num_labels - 1] hidden_to_data_weights; // Hidden[T] -> Labels. Base class gets 0.
  // ordered[n_H] hidden_bias[H]; // Use ordered if using NUTS
  row_vector[n_H] hidden_bias[H]; // Hidden layer biases
  row_vector[num_labels - 1] labels_bias; // Labels biases. Base class gets 0.
  
  vector[P] data_to_hidden_weights; // Data -> Hidden 1
  row_vector[num_labels + 1] y_bias; // Bias. 
  real<lower=0> sigma;
}

transformed parameters {
  vector[num_labels] output_layer_logit[N]; // Predicted output layer logits

  output_layer_logit = nn_predict(x,
                                  data_to_hidden_weights,
                                  y_bias);

}

model {
  // Priors
  to_vector(data_to_hidden_weights) ~ std_normal();

  for(h in 1:(H-1)) {
    to_vector(hidden_to_hidden_weights[h]) ~ std_normal();
  }

  to_vector(hidden_to_data_weights) ~ std_normal();

  for(h in 1:H) {
    to_vector(hidden_bias[h]) ~ std_normal();
  }
  labels_bias ~ std_normal();

  for(n in 1:N) { // Likelihood
    labels[n] ~ categorical_logit(output_layer_logit[n]);
  }
}

"""

In [2]:
# Put our data in a dictionary
data = {
          'N' :  len(all_feature_vec) ,# Number of training samples
          'P': len(all_feature_vec[0]),# Number of predictors (features)
          'x' : all_feature_vec, # Feature data
          'y' : [0]*len(nonwords)+[1]*len(words),  # Outcome
         
          'N_test' : 1, # Number of test samples
          'x_test' : all_feature_vec[0].reshape(1, 256), # Test predictors
        }


NameError: name 'all_feature_vec' is not defined

In [14]:

# Compile the model
sm = pystan.StanModel(model_code=stan_model)

# Train the model and generate samples
fit = sm.sampling(data=data, iter=100, chains=4, warmup=50, thin=1, seed=101)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_eacd95bfbc59a91eff2fa5ac198f1e91 NOW.
To run all diagnostics call pystan.check_hmc_diagnostics(fit)


In [15]:
summary_dict = fit.summary()
df = pd.DataFrame(summary_dict['summary'], 
                  columns=summary_dict['summary_colnames'], 
                  index=summary_dict['summary_rownames'])
print(df)


                                  mean   se_mean         sd         2.5%  \
data_to_hidden_weights[1]    -0.098133  0.037281   1.027804    -1.948176   
data_to_hidden_weights[2]    -0.075254  0.019479   1.044427    -1.911425   
data_to_hidden_weights[3]    -0.763150  0.020171   0.281909    -1.359367   
data_to_hidden_weights[4]     0.070524  0.026242   1.076125    -1.890631   
data_to_hidden_weights[5]     0.009106  0.036508   1.088426    -1.914638   
...                                ...       ...        ...          ...   
output_layer[3997]            0.992674  0.000289   0.003896     0.982899   
output_layer[3998]            0.460535  0.003921   0.061720     0.337265   
output_layer[3999]            0.008684  0.000480   0.007083     0.002299   
output_layer[4000]            0.773318  0.003810   0.052996     0.675669   
lp__                       2277.710700  1.239591  10.869477  2256.793026   

                                   25%          50%          75%        97.5%  \
data_t

/home/mahvash/.local/lib/python3.8/site-packages/pystan/misc.py:366: RuntimeWarning: invalid value encountered in sqrt
  sem=msd[:, 1] / np.sqrt(ess), ess=ess, rhat=rhat,


In [20]:
df.to_csv(r'./NNStanResults.txt', header=None, index=None, sep=' ', mode='a')


In [17]:
print(fit["sigma"])

[0.33850481 0.33096167 0.33160332 0.33298036 0.33001759 0.33763593
 0.32939929 0.33527001 0.33621886 0.33432167 0.33161017 0.33914696
 0.34264697 0.33861239 0.32938172 0.33589295 0.33595622 0.33710412
 0.3304404  0.33543409 0.33914307 0.33591074 0.33924723 0.3319405
 0.32913226 0.33387039 0.33256642 0.33718676 0.33831966 0.33454033
 0.34053651 0.33191031 0.33573597 0.34252111 0.33524132 0.33355408
 0.33302414 0.33309913 0.33785222 0.33237845 0.33730072 0.33169857
 0.34069931 0.3270495  0.33925463 0.33890173 0.33251578 0.33392747
 0.34068352 0.33761414 0.33598873 0.33118019 0.33702135 0.3376915
 0.33576115 0.3380666  0.33293769 0.341691   0.33529275 0.33512338
 0.33501594 0.33301234 0.33325987 0.33595064 0.33459481 0.33410302
 0.34218363 0.33096027 0.33523633 0.33511391 0.3327735  0.33581096
 0.33217555 0.33094496 0.33238213 0.33203774 0.33266741 0.33566552
 0.333877   0.3372037  0.3288915  0.33836759 0.33983196 0.33355527
 0.33683849 0.33870861 0.33714217 0.3388576  0.33536306 0.338475

### DDM Stan Simulator

In [ ]:
whole_model = """
data {
	int<lower=1> N;									// number of data items
	int<lower=1> K;									// number of options

	int<lower=-1,upper=1> accuracy[N];				// accuracy (-1, 1)
	real<lower=0> rt[N];							// rt

	vector[2] alpha_priors;							// mean and sd of the prior
	vector[2] drift_scaling_priors;					// mean and sd of the prior
	vector[2] threshold_priors;						// mean and sd of the prior
	vector[2] ndt_priors;							// mean and sd of the prior
	real<lower=0, upper=1> starting_point;			// starting point diffusion model not to estimate
}
transformed data {
	vector[K] Q0;
	Q0 = rep_vector(initial_value, K);
}
parameters {
	real alpha;
	real drift_scaling;
	real threshold;
	real ndt;
}
transformed parameters {
	real drift_ll[N];								// trial-by-trial drift rate for likelihood (incorporates accuracy)
	real drift_t[N];								// trial-by-trial drift rate for predictions
	real<lower=0> threshold_t[N];					// trial-by-trial threshold
	real<lower=0> ndt_t[N];							// trial-by-trial ndt

	vector[K] Q;									// Q state values

	real Q_mean;									// mean across all options
	real Q_mean_pres[N];							// mean Q presented options
	real delta_Q[N];								// Qcor - Qinc
	real PE_cor;									// prediction error correct option
	real PE_inc;									// prediction error incorrect option

	real transf_alpha;
	real transf_drift_scaling;
	real transf_threshold;
	real transf_ndt;

	transf_alpha = Phi(alpha);						// for the output
	transf_drift_scaling = log(1 + exp(drift_scaling));
	transf_threshold = log(1 + exp(threshold));
	transf_ndt = log(1 + exp(ndt));

	for (n in 1:N) {
		if (trial_block[n] == 1) {
			if (block_label[n] == 1) {
				Q = Q0;
			} else {
				Q_mean = mean(Q);
				Q = rep_vector(Q_mean, K);
			}
		}
		Q_mean_pres[n] = (Q[cor_option[n]] + Q[inc_option[n]])/2;
		delta_Q[n] = Q[cor_option[n]] - Q[inc_option[n]];
		PE_cor = f_cor[n] - Q[cor_option[n]];
		PE_inc = f_inc[n] - Q[inc_option[n]];

		drift_t[n] = transf_drift_scaling*delta_Q[n];
		drift_ll[n] = drift_t[n]*accuracy[n];
		threshold_t[n] = transf_threshold;
		ndt_t[n] = transf_ndt;

		Q[cor_option[n]] = Q[cor_option[n]] + transf_alpha*PE_cor;
		Q[inc_option[n]] = Q[inc_option[n]] + transf_alpha*PE_inc;
	}
}
model {
	alpha ~ normal(alpha_priors[1], alpha_priors[2]);
	drift_scaling ~ normal(drift_scaling_priors[1], drift_scaling_priors[2]);
	threshold ~ normal(threshold_priors[1], threshold_priors[2]);
	ndt ~ normal(ndt_priors[1], ndt_priors[2]);

	rt ~ wiener(threshold_t, ndt_t, starting_point, drift_ll);
}
generated quantities {
	vector[N] log_lik;

	{for (n in 1:N) {
		log_lik[n] = wiener_lpdf(rt[n] | threshold_t[n], ndt_t[n], starting_point, drift_ll[n]);
	}
	}
}
"""

In [13]:
import pystan
import nest_asyncio
nest_asyncio.apply()
model_code = 'parameters {real y;} model {y ~ normal(0,1);}'
model = pystan.StanModel(model_code=model_code)  # this will take a minute
y = model.sampling(n_jobs=1).extract()['y']
y.mean()  # should be close to 0

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_5944b02c79788fa0db5b3a93728ca2bf NOW.


TypeError: dist must be a Distribution instance

In [2]:
print(1)

1


In [18]:
import stan
import nest_asyncio
nest_asyncio.apply()

schools_code = """
data {
  int<lower=0> J;         // number of schools
  real y[J];              // estimated treatment effects
  real<lower=0> sigma[J]; // standard error of effect estimates
}
parameters {
  real mu;                // population treatment effect
  real<lower=0> tau;      // standard deviation in treatment effects
  vector[J] eta;          // unscaled deviation from mu by school
}
transformed parameters {
  vector[J] theta = mu + tau * eta;        // school treatment effects
}
model {
  target += normal_lpdf(eta | 0, 1);       // prior log-density
  target += normal_lpdf(y | theta, sigma); // log-likelihood
}
"""

In [19]:
schools_data = {"J": 8,
                "y": [28,  8, -3,  7, -1,  1, 18, 12],
                "sigma": [15, 10, 16, 11,  9, 11, 10, 18]}

In [ ]:
posterior = stan.build(schools_code, data=schools_data, random_seed=1)


INFO:aiohttp.access:127.0.0.1 [05/Dec/2021:10:49:51 +0000] "POST /v1/models/6lkgw2re/params HTTP/1.1" 404 249 "-" "Python/3.8 aiohttp/3.8.1"
INFO:httpstan:Building model-specific services extension module for `models/6lkgw2re`.


Building: 0.1s
Building: 0.2s
Building: 0.3s
Building: 0.4s
Building: 0.5s
Building: 0.6s
Building: 0.7s
Building: 0.8s
Building: 0.9s
Building: 1.0s
Building: 1.1s
Building: 1.3s
Building: 1.4s
Building: 1.5s
Building: 1.6s
Building: 1.7s
Building: 1.8s
Building: 1.9s
Building: 2.0s
Building: 2.1s
Building: 2.2s
Building: 2.3s
Building: 2.4s
Building: 2.5s
Building: 2.6s
Building: 2.7s
Building: 2.8s
Building: 2.9s
Building: 3.0s
Building: 3.1s
Building: 3.2s
Building: 3.3s
Building: 3.4s
Building: 3.5s
Building: 3.6s
Building: 3.7s
Building: 3.8s
Building: 3.9s
Building: 4.0s
Building: 4.1s
Building: 4.2s
Building: 4.3s
Building: 4.4s
Building: 4.5s
Building: 4.6s
Building: 4.7s
Building: 4.8s
Building: 4.9s
Building: 5.0s
Building: 5.1s
Building: 5.3s
Building: 5.4s
Building: 5.5s
Building: 5.6s
Building: 5.7s
Building: 5.8s
Building: 5.9s
Building: 6.0s
Building: 6.1s
Building: 6.2s
Building: 6.3s
Building: 6.4s
Building: 6.5s
Building: 6.6s
Building: 6.7s
Building: 6.8s
Building: 

Building: 26.5s
Building: 26.6s
Building: 26.7s
Building: 26.8s
Building: 26.9s
Building: 27.0s
Building: 27.1s
Building: 27.2s
Building: 27.3s
Building: 27.5s
Building: 27.6s
Building: 27.7s
Building: 27.8s
Building: 27.9s
Building: 28.0s
Building: 28.1s
Building: 28.2s
Building: 28.3s
Building: 28.4s
Building: 28.5s
Building: 28.6s
Building: 28.8s
Building: 28.9s
Building: 29.0s
Building: 29.1s
Building: 29.2s
Building: 29.3s
Building: 29.4s
Building: 29.5s
Building: 29.6s
Building: 29.7s
Building: 29.8s
Building: 29.9s
Building: 30.0s
Building: 30.2s
Building: 30.3s
Building: 30.4s
Building: 30.5s
Building: 30.6s
Building: 30.7s
Building: 30.8s
Building: 30.9s
Building: 31.0s
Building: 31.1s
Building: 31.2s
Building: 31.3s
Building: 31.5s
Building: 31.6s
Building: 31.7s
Building: 31.8s
Building: 31.9s
Building: 32.0s
Building: 32.1s
Building: 32.2s
Building: 32.3s
Building: 32.4s
Building: 32.6s
Building: 32.7s
Building: 32.8s
Building: 32.9s
Building: 33.0s
Building: 33.1s
Building

Building: 53.1s
Building: 53.2s
Building: 53.3s
Building: 53.4s
Building: 53.5s
Building: 53.6s
Building: 53.7s
Building: 53.8s
Building: 53.9s
Building: 54.0s
Building: 54.2s
Building: 54.3s
Building: 54.4s
Building: 54.5s
Building: 54.6s
Building: 54.7s
Building: 54.8s
Building: 54.9s
Building: 55.0s
Building: 55.1s
Building: 55.3s
Building: 55.4s
Building: 55.5s
Building: 55.6s
Building: 55.7s
Building: 55.8s
Building: 55.9s
Building: 56.0s
Building: 56.1s
Building: 56.2s
Building: 56.3s
Building: 56.4s
Building: 56.6s
Building: 56.7s
Building: 56.8s
Building: 56.9s
Building: 57.0s
Building: 57.1s
Building: 57.2s
Building: 57.3s
Building: 57.4s
Building: 57.5s
Building: 57.7s
Building: 57.8s
Building: 57.9s
Building: 58.0s
Building: 58.1s
Building: 58.2s
Building: 58.3s
Building: 58.4s
Building: 58.5s
Building: 58.6s
Building: 58.7s
Building: 58.8s
Building: 59.0s
Building: 59.1s
Building: 59.2s
Building: 59.3s
Building: 59.4s
Building: 59.5s
Building: 59.6s
Building: 59.7s
Building

Building: 80.0s
Building: 80.1s
Building: 80.2s
Building: 80.3s
Building: 80.4s
Building: 80.5s
Building: 80.6s
Building: 80.7s
Building: 80.8s
Building: 80.9s
Building: 81.0s
Building: 81.2s
Building: 81.3s
Building: 81.4s
Building: 81.5s
Building: 81.6s
Building: 81.7s
Building: 81.8s
Building: 81.9s
Building: 82.0s
Building: 82.1s
Building: 82.3s
Building: 82.4s
Building: 82.5s
Building: 82.6s
Building: 82.7s
Building: 82.8s
Building: 82.9s
Building: 83.0s
Building: 83.1s
Building: 83.2s
Building: 83.3s
Building: 83.4s
Building: 83.5s
Building: 83.7s
Building: 83.8s
Building: 83.9s
Building: 84.0s
Building: 84.1s
Building: 84.2s
Building: 84.3s
Building: 84.4s
Building: 84.5s
Building: 84.6s
Building: 84.7s
Building: 84.9s
Building: 85.0s
Building: 85.1s
Building: 85.2s
Building: 85.3s
Building: 85.4s
Building: 85.5s
Building: 85.6s
Building: 85.7s
Building: 85.8s
Building: 85.9s
Building: 86.0s
Building: 86.2s
Building: 86.3s
Building: 86.4s
Building: 86.5s
Building: 86.6s
Building

...

Building: 132.4s
Building: 132.5s
Building: 132.7s
Building: 132.8s
Building: 132.9s
Building: 133.0s
Building: 133.1s
Building: 133.2s
Building: 133.3s
Building: 133.4s
Building: 133.5s
Building: 133.6s
Building: 133.7s
Building: 133.9s
Building: 134.0s
Building: 134.1s
Building: 134.2s
Building: 134.3s
Building: 134.4s
Building: 134.5s
Building: 134.6s
Building: 134.7s
Building: 134.8s
Building: 134.9s
Building: 135.1s
Building: 135.2s
Building: 135.3s
Building: 135.4s
Building: 135.5s
Building: 135.6s
Building: 135.7s
Building: 135.8s
Building: 135.9s
Building: 136.0s
Building: 136.1s
Building: 136.2s
Building: 136.3s
Building: 136.5s
Building: 136.6s
Building: 136.7s
Building: 136.8s
Building: 136.9s
Building: 137.0s
Building: 137.1s
Building: 137.2s
Building: 137.3s
Building: 137.4s
Building: 137.5s
Building: 137.6s
Building: 137.8s
Building: 137.9s
Building: 138.0s
Building: 138.1s
Building: 138.2s
Building: 138.3s
Building: 138.4s
Building: 138.5s
Building: 138.6s
Building: 138.

Building: 158.4s
Building: 158.5s
Building: 158.6s
Building: 158.7s
Building: 158.8s
Building: 158.9s
Building: 159.0s
Building: 159.2s
Building: 159.3s
Building: 159.4s
Building: 159.5s
Building: 159.6s
Building: 159.7s
Building: 159.8s
Building: 159.9s
Building: 160.0s
Building: 160.1s
Building: 160.2s
Building: 160.4s
Building: 160.5s
Building: 160.6s
Building: 160.7s
Building: 160.8s
Building: 160.9s
Building: 161.0s
Building: 161.1s
Building: 161.2s
Building: 161.3s
Building: 161.4s
Building: 161.6s
Building: 161.7s
Building: 161.8s
Building: 161.9s
Building: 162.0s
Building: 162.1s
Building: 162.2s
Building: 162.3s
Building: 162.4s
Building: 162.5s
Building: 162.6s
Building: 162.8s
Building: 162.9s
Building: 163.0s
Building: 163.1s
Building: 163.2s
Building: 163.3s
Building: 163.4s
Building: 163.5s
Building: 163.6s
Building: 163.7s
Building: 163.8s
Building: 163.9s
Building: 164.1s
Building: 164.2s
Building: 164.3s
Building: 164.4s
Building: 164.5s
Building: 164.6s
Building: 164.

Building: 184.3s
Building: 184.5s
Building: 184.6s
Building: 184.7s
Building: 184.8s
Building: 184.9s
Building: 185.0s
Building: 185.1s
Building: 185.2s
Building: 185.3s
Building: 185.4s
Building: 185.6s
Building: 185.7s
Building: 185.8s
Building: 185.9s
Building: 186.0s
Building: 186.1s
Building: 186.2s
Building: 186.3s
Building: 186.4s
Building: 186.5s
Building: 186.7s
Building: 186.8s
Building: 186.9s
Building: 187.0s
Building: 187.1s
Building: 187.2s
Building: 187.3s
Building: 187.4s
Building: 187.5s
Building: 187.7s
Building: 187.8s
Building: 187.9s
Building: 188.0s
Building: 188.1s
Building: 188.2s
Building: 188.3s
Building: 188.4s
Building: 188.5s
Building: 188.6s
Building: 188.7s
Building: 188.8s
Building: 189.0s
Building: 189.1s
Building: 189.2s
Building: 189.3s
Building: 189.4s
Building: 189.5s
Building: 189.6s
Building: 189.7s
Building: 189.8s
Building: 189.9s
Building: 190.0s
Building: 190.1s
Building: 190.3s
Building: 190.4s
Building: 190.5s
Building: 190.6s
Building: 190.

Building: 210.4s
Building: 210.5s
Building: 210.6s
Building: 210.7s
Building: 210.8s
Building: 210.9s
Building: 211.0s
Building: 211.1s
Building: 211.2s
Building: 211.4s
Building: 211.5s
Building: 211.6s
Building: 211.7s
Building: 211.8s
Building: 211.9s
Building: 212.0s
Building: 212.1s
Building: 212.2s
Building: 212.3s
Building: 212.5s
Building: 212.6s
Building: 212.7s
Building: 212.8s
Building: 212.9s
Building: 213.0s
Building: 213.1s
Building: 213.2s
Building: 213.3s
Building: 213.4s
Building: 213.6s
Building: 213.7s
Building: 213.8s
Building: 213.9s
Building: 214.0s
Building: 214.1s
Building: 214.2s
Building: 214.3s
Building: 214.4s
Building: 214.6s
Building: 214.7s
Building: 214.8s
Building: 214.9s
Building: 215.0s
Building: 215.1s
Building: 215.2s
Building: 215.3s
Building: 215.4s
Building: 215.5s
Building: 215.7s
Building: 215.8s
Building: 215.9s
Building: 216.0s
Building: 216.1s
Building: 216.2s
Building: 216.3s
Building: 216.4s
Building: 216.5s
Building: 216.6s
Building: 216.

Building: 236.4s
Building: 236.5s
Building: 236.6s
Building: 236.7s
Building: 236.8s
Building: 237.0s
Building: 237.1s
Building: 237.2s
Building: 237.3s
Building: 237.4s
Building: 237.5s
Building: 237.6s
Building: 237.7s
Building: 237.8s
Building: 237.9s
Building: 238.1s
Building: 238.2s
Building: 238.3s
Building: 238.4s
Building: 238.5s
Building: 238.6s
Building: 238.7s
Building: 238.8s
Building: 238.9s
Building: 239.1s
Building: 239.2s
Building: 239.3s
Building: 239.4s
Building: 239.5s
Building: 239.6s
Building: 239.7s
Building: 239.8s
Building: 239.9s
Building: 240.0s
Building: 240.2s
Building: 240.3s
Building: 240.4s
Building: 240.5s
Building: 240.6s
Building: 240.7s
Building: 240.8s
Building: 240.9s
Building: 241.0s
Building: 241.1s
Building: 241.3s
Building: 241.4s
Building: 241.5s
Building: 241.6s
Building: 241.7s
Building: 241.8s
Building: 241.9s
Building: 242.0s
Building: 242.1s
Building: 242.2s
Building: 242.4s
Building: 242.5s
Building: 242.6s
Building: 242.7s
Building: 242.

In [17]:
fit = posterior.sample(num_chains=4, num_samples=1000)


NameError: name 'posterior' is not defined

In [ ]:
eta = fit["eta"]  # array with shape (8, 4000)


In [ ]:
df = fit.to_frame()
